In [17]:
import helpers
from helpers import *

In [18]:
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

# W2

In [19]:
qreg = QuantumRegister(2)
w2 = QuantumCircuit(qreg)
w2.x(qreg[0])
Bdirect(w2, qreg, 1, 0, 0.5)
print(w2)
job = qiskit.execute(w2, Aer.get_backend('statevector_simulator'))
theorical_psi = job.result().get_statevector(w2)
print(theorical_psi)

#actual circuit using not-so-bad qbits (9&10)
qreg = QuantumRegister(14)
w2 = QuantumCircuit(qreg)
w2.x(qreg[9])
Balt(w2, qreg, 10, 9, 0.5)
w2_tomo_circuit = state_tomography_circuits(w2, [qreg[9], qreg[10]])
print(w2_tomo_circuit[0])
job = qiskit.execute(w2_tomo_circuit, Aer.get_backend('qasm_simulator'), shots=8000)
fitter = StateTomographyFitter(job.result(), w2_tomo_circuit)
fitted_rho = fitter.fit()
fidelity = state_fidelity(theorical_psi, fitted_rho)
print(fidelity)


                ┌───┐                  ┌───┐                 ┌───┐
q6_0: |0>───────┤ X ├─────────■────────┤ H ├──────────────■──┤ H ├
         ┌──────┴───┴──────┐┌─┴─┐┌─────┴───┴──────┐┌───┐┌─┴─┐├───┤
q6_1: |0>┤ U3(-0.7854,0,0) ├┤ X ├┤ U3(0.7854,0,0) ├┤ H ├┤ X ├┤ H ├
         └─────────────────┘└───┘└────────────────┘└───┘└───┘└───┘
[ 0.        +0.j -0.70710678+0.j  0.70710678+0.j  0.        +0.j]
                                                                    ░         »
 q7_0: |0>──────────────────────────────────────────────────────────░─────────»
                                                                    ░         »
 q7_1: |0>──────────────────────────────────────────────────────────░─────────»
                                                                    ░         »
 q7_2: |0>──────────────────────────────────────────────────────────░─────────»
                                                                    ░         »
 q7_3: |0>─────────────────────────────

In [20]:
gate_times = [
        ('u1', None, 0), ('u2', None, 100), ('u3', None, 200),
        ('cx', [1, 0], 678), ('cx', [1, 2], 547), ('cx', [2, 3], 721),
        ('cx', [4, 3], 733), ('cx', [4, 10], 721), ('cx', [5, 4], 800),
        ('cx', [5, 6], 800), ('cx', [5, 9], 895), ('cx', [6, 8], 895),
        ('cx', [7, 8], 640), ('cx', [9, 8], 895), ('cx', [9, 10], 800),
        ('cx', [11, 10], 721), ('cx', [11, 3], 634), ('cx', [12, 2], 773),
        ('cx', [13, 1], 2286), ('cx', [13, 12], 1504), ('cx', [], 800)
    ]

noise_model = noise.device.basic_device_noise_model(melbourne.properties(), gate_times=gate_times)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'u2', 'u3']
  Instructions with noise: ['u3', 'u2', 'cx', 'measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
  Specific qubit errors: [('u2', [0]), ('u2', [1]), ('u2', [2]), ('u2', [3]), ('u2', [4]), ('u2', [5]), ('u2', [6]), ('u2', [7]), ('u2', [8]), ('u2', [9]), ('u2', [10]), ('u2', [11]), ('u2', [12]), ('u2', [13]), ('u3', [0]), ('u3', [1]), ('u3', [2]), ('u3', [3]), ('u3', [4]), ('u3', [5]), ('u3', [6]), ('u3', [7]), ('u3', [8]), ('u3', [9]), ('u3', [10]), ('u3', [11]), ('u3', [12]), ('u3', [13]), ('cx', [1, 0]), ('cx', [1, 2]), ('cx', [2, 3]), ('cx', [4, 3]), ('cx', [4, 10]), ('cx', [5, 4]), ('cx', [5, 6]), ('cx', [5, 9]), ('cx', [6, 8]), ('cx', [7, 8]), ('cx', [9, 8]), ('cx', [9, 10]), ('cx', [11, 3]), ('cx', [11, 10]), ('cx', [11, 12]), ('cx', [12, 2]), ('cx', [13, 1]), ('cx', [13, 12]), ('measure', [0]), ('measure', [1]), ('measure', [2]), ('measure', [3]), ('measure', [4]), ('measure', [5]), ('measure', [6]), 

In [21]:
meas_calibs, state_labels = mc.complete_meas_cal(qubit_list=[9, 10])

backend = Aer.get_backend('qasm_simulator')

qobj_cal = qiskit.compile(meas_calibs, backend=backend, shots=15000)
qobj_tomo = qiskit.compile(w2_tomo_circuit, backend=backend, shots=15000)

job_cal = backend.run(qobj_cal, noise_model=noise_model)
job_tomo = backend.run(qobj_tomo, noise_model=noise_model)

meas_fitter = mc.CompleteMeasFitter(job_cal.result(),state_labels)

fitter = StateTomographyFitter(job_tomo.result(), w2_tomo_circuit)

#no correction
fitted_rho = fitter.fit()
fidelity = state_fidelity(theorical_psi, fitted_rho)
print('Fit Fidelity (no correction) =', fidelity)

#correct data
correct_tomo_results = meas_fitter.filter.apply(job_tomo.result())
fitter = StateTomographyFitter(correct_tomo_results, w2_tomo_circuit)
fitted_rho = fitter.fit()
fidelity = state_fidelity(theorical_psi, fitted_rho)
print('Fit Fidelity (w/ correction) =', fidelity)

D:\Programmes\Anaconda3\envs\master_pds\lib\site-packages\qiskit\tools\compiler.py:57: DeprecationWarning: qiskit.compile() is deprecated and will be removed in Qiskit Terra 0.9. Please use qiskit.compiler.transpile() to transform circuits and qiskit.compiler.assemble() to produce a runnable qobj.
  DeprecationWarning)
D:\Programmes\Anaconda3\envs\master_pds\lib\site-packages\qiskit\tools\compiler.py:57: DeprecationWarning: qiskit.compile() is deprecated and will be removed in Qiskit Terra 0.9. Please use qiskit.compiler.transpile() to transform circuits and qiskit.compiler.assemble() to produce a runnable qobj.
  DeprecationWarning)


Fit Fidelity (no correction) = 0.8263963426830249
Fit Fidelity (w/ correction) = 0.9460398044902382


In [22]:
print(fitted_rho)
print(theorical_psi)

[[ 1.56974518e-02+0.00000000e+00j -2.07396408e-05+4.22594501e-04j
   5.83027814e-03-9.78631166e-05j -9.75739946e-03-1.19143714e-03j]
 [-2.07396408e-05-4.22594501e-04j  4.91288569e-01+0.00000000e+00j
  -4.61851682e-01-6.76794643e-04j  2.27379953e-03-1.72429722e-03j]
 [ 5.83027814e-03+9.78631166e-05j -4.61851682e-01+6.76794643e-04j
   4.77087676e-01+0.00000000e+00j  6.80548266e-03-6.38034577e-04j]
 [-9.75739946e-03+1.19143714e-03j  2.27379953e-03+1.72429722e-03j
   6.80548266e-03+6.38034577e-04j  1.59263023e-02+0.00000000e+00j]]
[ 0.        +0.j -0.70710678+0.j  0.70710678+0.j  0.        +0.j]


#### sim vs device

In [23]:
meas_calibs, state_labels = mc.complete_meas_cal(qubit_list=[9, 10])

backend = melbourne

qobj_cal = qiskit.compile(meas_calibs, backend=backend, shots=1000)
qobj_tomo = qiskit.compile(w2_tomo_circuit, backend=backend, shots=1000)

job_cal = backend.run(qobj_cal)
job_tomo = backend.run(qobj_tomo)

job_monitor(job_cal, monitor_async=True)
job_monitor(job_tomo, monitor_async=True)

D:\Programmes\Anaconda3\envs\master_pds\lib\site-packages\qiskit\tools\compiler.py:57: DeprecationWarning: qiskit.compile() is deprecated and will be removed in Qiskit Terra 0.9. Please use qiskit.compiler.transpile() to transform circuits and qiskit.compiler.assemble() to produce a runnable qobj.
  DeprecationWarning)
D:\Programmes\Anaconda3\envs\master_pds\lib\site-packages\qiskit\tools\compiler.py:57: DeprecationWarning: qiskit.compile() is deprecated and will be removed in Qiskit Terra 0.9. Please use qiskit.compiler.transpile() to transform circuits and qiskit.compiler.assemble() to produce a runnable qobj.
  DeprecationWarning)


HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

In [24]:
cal_results = job_cal.result()
tomo_results = job_tomo.result()

In [26]:
meas_fitter = mc.CompleteMeasFitter(cal_results,state_labels)

fitter = StateTomographyFitter(tomo_results, w2_tomo_circuit)

#no correction
fitted_rho = fitter.fit()
fidelity = state_fidelity(theorical_psi, fitted_rho)
print('Fit Fidelity (no correction) =', fidelity)

#correct data
correct_tomo_results = meas_fitter.filter.apply(job_tomo.result())
fitter = StateTomographyFitter(correct_tomo_results, w2_tomo_circuit)
fitted_rho = fitter.fit()
fidelity = state_fidelity(theorical_psi, fitted_rho)
print('Fit Fidelity (w/ correction) =', fidelity)

Fit Fidelity (no correction) = 0.7485447292384284
Fit Fidelity (w/ correction) = 0.8665055391714731
